In [0]:
!curl https://colab.chainer.org/install | sh -

!apt-get install graphviz
!pip install 'chaineripy'
!pip install 'chainercv'

import sys
import os
from functools import partial
import shutil
import json

from chainer.links.caffe.caffe_function import CaffeFunction
from chainer.serializers import npz
from chainer.datasets import cifar, split_dataset_random, TransformDataset
from chainer import links as L
from chainer import initializers, serializers
from google.colab import drive
import numpy as np


base_dir = './drive/My Drive/study'
sys.path.append(base_dir)
drive.mount('/content/drive')
caffemodel = CaffeFunction(base_dir + '/squeezenet/caffe/squeezenet_v1.1.caffemodel')
npz.save_npz('squeezenet_v1_1.npz', caffemodel, compression=False)
caffemodel = CaffeFunction(base_dir + '/squeezenet/caffe/squeezeNet_residual.caffemodel')
npz.save_npz('squeeze_resnet.npz', caffemodel, compression=False)

from resnet.networks.resnet101 import ResNet101
from resnet.networks.resnet152 import ResNet152
from squeezenet.networks.squeezenet import SqueezeNet
from squeezenet.networks.squeezenet_bn import SqueezeNetBN
from squeezenet.networks.squeeze_resnet import SqueezeResNet
from squeezenet.networks.squeeze_pre_resnet import SqueezePreResNet
from distill.knowledge_distill import DistillClassifier, softmax_cross_entropy_softlabel
from distill.utils import save_softlabels, generate_softlabel
from execute import run_train
from transform import transform, transform_with_softlabel

# Dataset setup
train_val, test = cifar.get_cifar10(scale=255.)
train_size      = int(len(train_val) * 0.9)
train, valid    = split_dataset_random(train_val, train_size, seed=0)
mean = np.mean([x for x, _ in train], axis=(0, 2, 3))
std  = np.std([x for x, _ in train], axis=(0, 2, 3))

params = json.load(open(base_dir + '/params.json', "r"))
distill_params = json.load(open(base_dir + '/distill/distill_params.json', "r"))
train_soft_path = base_dir + '/soft_labels.npy'
valid_soft_path = base_dir + '/soft_labels_valid.npy'

## Generating soft labels

In [0]:
teacher = ResNet101(10, pretrained_model=None)
serializers.load_npz(
    'drive/My Drive/study/result/snapshot_epoch-33',
    teacher, path='updater/model:main/predictor/')
teacher.to_gpu()
fun_generate_soft = partial(generate_softlabel, model=teacher, mean=mean, std=std, **params)

if not os.path.exists(train_soft_path):
    save_softlabels(train, fun_generate_soft, base_dir + '/soft_labels')
if not os.path.exists(valid_soft_path):
    save_softlabels(valid, fun_generate_soft, base_dir + '/soft_labels_valid')

## Main

In [24]:
from chainer.dataset.convert import concat_examples
from chainer.datasets import tuple_dataset

img_t, lab_t = concat_examples(train)
soft_labels_t = np.load(base_dir + '/soft_labels.npy')
train_soft = tuple_dataset.TupleDataset(img_t, soft_labels_t, lab_t)

img_v, lab_v = concat_examples(valid)
soft_labels_v = np.load(base_dir + '/soft_labels_valid.npy')
valid_soft = tuple_dataset.TupleDataset(img_v, soft_labels_v, lab_v)

# DATA AUGMENTATION
transform_train = partial(transform_with_softlabel, mean=mean, std=std, train=True, **params)
transform_valid = partial(transform_with_softlabel, mean=mean, std=std, train=False, **params)
processed_train = TransformDataset(train_soft, transform_train)
processed_valid = TransformDataset(valid_soft, transform_valid)

# MODEL DEFINITION AND FINETUNING
student = SqueezeNet(10, init_param=initializers.HeNormal())
serializers.load_npz(
    'drive/My Drive/study/result/snapshot_epoch-40',
    student, path='updater/model:main/predictor/')

# DISTILL
alpha = distill_params['alpha']
t = distill_params['temperature']
net = DistillClassifier(student, lossfun_soft=softmax_cross_entropy_softlabel, alpha=alpha, t=t)


# RUN!
params['save_trainer_interval'] = 6
params['batchsize']     = 128
params['result_dir']    = 'drive/My Drive/study/result'
run_train(processed_train, processed_valid, net, **params)

epoch       main/loss   main/accuracy  val/main/loss  val/main/accuracy  lr          elapsed_time
1           24.8915     0.139138       23.1447        0.193359           0.001       175.15        
2           21.9046     0.2504         18.4107        0.345508           0.001       351.456       
3           18.8533     0.350739       15.8025        0.45625            0.001       527.89        
4           15.4474     0.486728       12.8443        0.574023           0.001       705.171       
5           12.3497     0.609731       10.6911        0.674609           0.001       882.474       
6           9.55761     0.711648       7.22654        0.786914           0.001       1060.41       
7           7.53711     0.780783       5.84999        0.838086           0.001       1238.89       
8           6.56787     0.81068        5.3367         0.857813           0.001       1416.59       
9           6.03087     0.826682       4.80809        0.874219           0.001       1594.38       
10